# Run this first.

In [5]:
import matplotlib
matplotlib.use("QtAgg") # set matplotlib backend

import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, date, timedelta
import textwrap

logFolder = r'C:\Users\Jaseung\Downloads\logfiles'
# logFolder = r'/home/jaseung/Downloads/logfiles'

def load_temperature_oneday(date:date):
    """
        Read temperature log files and return two pandas dataframe of datetime and temperatures.
    """
    
    date_str = date.strftime("%y-%m-%d")
    base_path = os.path.join(logFolder, date_str)    
    file_names = ['CH1 T ' + date_str + '.log', 
                  'CH2 T ' + date_str + '.log', 
                  'CH5 T ' + date_str + '.log', 
                  'CH6 T ' + date_str + '.log' ]

    full_file_names = [ os.path.join(base_path, file_name) for file_name in file_names ]

    df_datetimes, df_temperatures = pd.DataFrame(), pd.DataFrame()
    
    for file_name in full_file_names:
        with open(file_name, 'r') as f: 
            df = pd.read_csv(f, names=['date', 'time', 'temperature'], header=0)
            df_datetime  = pd.to_datetime(df['date'] + df['time'], format=' %d-%m-%y%H:%M:%S')
            df_datetimes = pd.concat([df_datetimes, df_datetime], axis=1)             
            df_temperatures = pd.concat([df_temperatures, df['temperature']], axis=1)
           
#     print(df_datetimes.shape)
#     print(df_temperatures.shape)   
    
    return df_datetimes, df_temperatures

def load_pressure_oneday(date:date):
    """
        Read pressure log file and return two pandas dataframe of datetime and pressure.
    """
    date_str = date.strftime("%y-%m-%d")
    base_path = os.path.join(logFolder, date_str)  
    file_name = 'maxigauge ' + date_str + '.log'
    full_file_name = os.path.join(base_path, file_name)

    df_datetimes, df_pressures = pd.DataFrame(), pd.DataFrame()
    with open(full_file_name, 'r') as f:
        df = pd.read_csv(f, header=None)
        df_datetimes  = pd.to_datetime(df.iloc[:,0] + df.iloc[:,1], format='%d-%m-%y%H:%M:%S')
        df_pressures = df.iloc[:, [5,11,17,23,29,35]]
       
    # print(df_datetimes.shape)
    # print(df_pressures.shape)   

    return df_datetimes, df_pressures

def load_flowmeter_oneday(date: date):
    """
        Read flowmeter log file and return two pandas dataframe of datetime and flowrate.
    """
    date_str = date.strftime("%y-%m-%d")
    base_path = os.path.join(logFolder, date_str)  
    file_name = 'Flowmeter ' + date_str + '.log'
    full_file_name = os.path.join(base_path, file_name)

    df_datetimes, df_flowmeter = pd.DataFrame(), pd.DataFrame()
    with open(full_file_name, 'r') as f:
        df = pd.read_csv(f, header=None)
        df_datetimes  = pd.to_datetime(df.iloc[:,0] + df.iloc[:,1], format=' %d-%m-%y%H:%M:%S')
        df_flowmeter = df.iloc[:,2]
       
    # print(df_datetimes.shape)
    # print(df_flowmeter.shape)   

    return df_datetimes, df_flowmeter

def load_status_oneday(date:date):
    """
    Read status log file and return two pandas dataframe of datetime and status.
    """
    date_str = date.strftime("%y-%m-%d")
    base_path = os.path.join(logFolder, date_str)  
    file_name = 'Status_' + date_str + '.log'
    full_file_name = os.path.join(base_path, file_name)

    df_datetimes, df_status = pd.DataFrame(), pd.DataFrame()
    with open(full_file_name, 'r') as f:
        df = pd.read_csv(f, header=None)
        df_datetimes  = pd.to_datetime(df.iloc[:,0] + df.iloc[:,1], format='%d-%m-%y%H:%M:%S')
        df_status = df.iloc[:, list(np.arange(3,51,2))]
        
    # print(df_datetimes.shape)
    # print(df_pressures.shape)   

    return df_datetimes, df_status


def load_temperature(start_date:date, end_date:date):
    """
    Return time and temperature dataframes between start_date and end_date
    """
    df_datetimes_all, df_temperatures_all = pd.DataFrame(), pd.DataFrame()
    
    while start_date <= end_date:
        
        try:
            df_datetimes, df_temperatures = load_temperature_oneday(start_date)
        except FileNotFoundError:
            df_datetimes, df_temperatures = pd.DataFrame(), pd.DataFrame()
            print(f"FileNotFound: {start_date}, temperature")

        df_datetimes_all = pd.concat([df_datetimes_all, df_datetimes], axis=0)
        df_temperatures_all = pd.concat([df_temperatures_all, df_temperatures], axis=0)
        
        start_date += timedelta(days=1)
    
    return df_datetimes_all, df_temperatures_all
        
def load_pressure(start_date:date, end_date:date):
    """
    Return time and pressure dataframes between start_date and end_date
    """
    df_datetimes_all, df_pressures_all = pd.DataFrame(), pd.DataFrame()
    while start_date <= end_date:
        try:
            df_datetimes, df_pressures = load_pressure_oneday(start_date)
        except FileNotFoundError:
            df_datetimes, df_pressures = pd.DataFrame(), pd.DataFrame()
            print(f"FileNotFound: {start_date}, pressure")

        df_datetimes_all = pd.concat([df_datetimes_all, df_datetimes], axis=0)
        df_pressures_all = pd.concat([df_pressures_all, df_pressures], axis=0)
        
        start_date += timedelta(days=1)
    
    return df_datetimes_all, df_pressures_all

def load_flowmeter(start_date: date, end_date: date):
    """
    Return time and flowmeter dataframes between start_date and end_date
    """
    df_datetimes_all, df_flowmeters_all = pd.DataFrame(), pd.DataFrame()
    while start_date <= end_date:
        try:
            df_datetimes, df_flowmeters = load_flowmeter_oneday(start_date)
        except FileNotFoundError:
            df_datetimes, df_flowmeters = pd.DataFrame(), pd.DataFrame()
            print(f"FileNotFound: {start_date}, flowmeter")
    
        df_datetimes_all = pd.concat([df_datetimes_all, df_datetimes], axis=0)
        df_flowmeters_all = pd.concat([df_flowmeters_all, df_flowmeters], axis=0)
        
        start_date += timedelta(days=1)
    
    return df_datetimes_all, df_flowmeters_all

def load_status(start_date:date, end_date:date):
    """
    Return time and status dataframes between start_date and end_date
    """
    df_datetimes_all, df_status_all = pd.DataFrame(), pd.DataFrame()
    while start_date <= end_date:
        try:
            df_datetimes, df_status = load_status_oneday(start_date)
        except FileNotFoundError:
            df_datetimes, df_status = pd.DataFrame(), pd.DataFrame()
            print(f"FileNotFound: {start_date}, status")

        df_datetimes_all = pd.concat([df_datetimes_all, df_datetimes], axis=0)
        df_status_all = pd.concat([df_status_all, df_status], axis=0)
        
        start_date += timedelta(days=1)
    
    return df_datetimes_all, df_status_all


def plot_temperature(start_date: date, end_date: date):
    """
    Plot temperature.
    """
    df_df_datetimes_all, df_temperatures_all = load_temperature(start_date, end_date)

    fig, ax = plt.subplots()
    
    text = "Temperature from " + start_date.strftime("%y-%m-%d") + " to " + end_date.strftime("%y-%m-%d")
    title = "Click on legend line to toggle line on/off" + "\n" + "\n".join(textwrap.wrap(text, 60))
    ax.set_title(title)
    ax.set_xlabel('Datetime')
    ax.set_ylabel('Temperature(K)')
    ax.grid()
    
    line1, = ax.plot(df_df_datetimes_all.iloc[:,0], df_temperatures_all.iloc[:,0], label="50 K")
    line2, = ax.plot(df_df_datetimes_all.iloc[:,1], df_temperatures_all.iloc[:,1], label="4 K")
    line3, = ax.plot(df_df_datetimes_all.iloc[:,2], df_temperatures_all.iloc[:,2], label="Still")
    line4, = ax.plot(df_df_datetimes_all.iloc[:,3], df_temperatures_all.iloc[:,3], label="MCX")
    leg = ax.legend(fancybox=True, shadow=True)
    
    lines = [line1, line2, line3, line4]
    lined = {}  # Will map legend lines to original lines.
    for legline, origline in zip(leg.get_lines(), lines):
        legline.set_picker(True)  # Enable picking on the legend line.
        lined[legline] = origline
    
    def on_pick(event):
        # On the pick event, find the original line corresponding to the legend
        # proxy line, and toggle its visibility.
        legline = event.artist
        origline = lined[legline]
        visible = not origline.get_visible()
        origline.set_visible(visible)
        # Change the alpha on the line in the legend so we can see what lines
        # have been toggled.
        legline.set_alpha(1.0 if visible else 0.2)
        fig.canvas.draw()

    fig.canvas.mpl_connect('pick_event', on_pick)
    plt.show(block=False)

    return fig, ax   

def plot_pressure(start_date: date, end_date: date):
    """
    Plot pressure
    """
    df_datetimes_all, df_pressures_all = load_pressure(start_date, end_date)

    fig, ax = plt.subplots()
    
    text = "Pressures from " + start_date.strftime("%y-%m-%d") + " to " + end_date.strftime("%y-%m-%d")
    title = "Click on legend line to toggle line on/off" + "\n" + "\n".join(textwrap.wrap(text, 60))
    ax.set_title(title)
    ax.set_xlabel('Datetime')
    ax.set_ylabel('Pressure (mBar)')
    ax.grid()
    
    line1, = ax.plot(df_datetimes_all, df_pressures_all.iloc[:,0], label="P1")
    line2, = ax.plot(df_datetimes_all, df_pressures_all.iloc[:,1], label="P2")
    line3, = ax.plot(df_datetimes_all, df_pressures_all.iloc[:,2], label="P3")
    line4, = ax.plot(df_datetimes_all, df_pressures_all.iloc[:,3], label="P4")
    line5, = ax.plot(df_datetimes_all, df_pressures_all.iloc[:,4], label="P5")
    line6, = ax.plot(df_datetimes_all, df_pressures_all.iloc[:,5], label="P6")
    leg = ax.legend(fancybox=True, shadow=True)
    
    lines = [line1, line2, line3, line4, line5, line6]
    lined = {}  # Will map legend lines to original lines.
    for legline, origline in zip(leg.get_lines(), lines):
        legline.set_picker(True)  # Enable picking on the legend line.
        lined[legline] = origline
    
    def on_pick(event):
        # On the pick event, find the original line corresponding to the legend
        # proxy line, and toggle its visibility.
        legline = event.artist
        origline = lined[legline]
        visible = not origline.get_visible()
        origline.set_visible(visible)
        # Change the alpha on the line in the legend so we can see what lines
        # have been toggled.
        legline.set_alpha(1.0 if visible else 0.2)
        fig.canvas.draw()

    fig.canvas.mpl_connect('pick_event', on_pick)
    plt.show(block=False)

def plot_flowmeter(start_date: date, end_date: date):
    """
    Plot flowmeter.
    """
    df_datetimes_all, df_flowmeters_all = load_flowmeter(start_date, end_date)

    fig, ax = plt.subplots()
    
    text = "Flowmeter from " + start_date.strftime("%y-%m-%d") + " to " + end_date.strftime("%y-%m-%d")
    title = "Click on legend line to toggle line on/off" + "\n" + "\n".join(textwrap.wrap(text, 60))
    ax.set_title(title)
    ax.set_xlabel('Datetime')
    ax.set_ylabel('Flowrate (mmol/s)')
    ax.grid()
    
    line1, = ax.plot(df_datetimes_all, df_flowmeters_all.iloc[:,0], label="Flowmeter")
    leg = ax.legend(fancybox=True, shadow=True)
    plt.show(block=False)

def plot_status(start_date: date, end_date: date):
    """
    Plot status.
    """
    df_datetimes_all, df_status_all = load_status(start_date, end_date)

    fig, ax = plt.subplots()
    
    text = "Status from " + start_date.strftime("%y-%m-%d") + " to " + end_date.strftime("%y-%m-%d")
    title = "Click on legend line to toggle line on/off" + "\n" + "\n".join(textwrap.wrap(text, 60))
    ax.set_title(title)
    ax.set_xlabel('Datetime')
    ax.set_ylabel('Status')
    ax.grid()
    

    s = "01-01-22,00:14:17,cptempwi,1.450000E+1,cptempwo,2.480000E+1,cptemph,7.050000E+1,cptempo,3.090000E+1,cpttime,2.548976E+6,cperrcode,2.800000E+1,cpavgl,8.840000E+1,cpavgh,2.748000E+2,nxdsf,3.000000E+1,nxdsct,4.100000E+1,nxdst,4.910500E+4,nxdsbs,4.050300E+4,nxdstrs,2.185700E+4,tc400remoteprio,1.000000E+0,tc400spdswptatt,1.000000E+0,tc400errorcode,0.000000E+0,tc400ovtempelec,0.000000E+0,tc400ovtemppump,0.000000E+0,tc400setspdatt,1.000000E+0,tc400pumpaccel,0.000000E+0,tc400heating,0.000000E+0,tc400standby,0.000000E+0,tc400pumpstatn,1.000000E+0,tc400commerr,0.000000E+0"
    legend_labels = s.split(',')[2::2]
    
    lines = []
    for i in range(24):
        line, = ax.plot(df_datetimes_all, df_status_all.iloc[:,i], label=legend_labels[i])
        lines.append(line)

    leg = ax.legend(fancybox=True, shadow=True)
    
    lined = {}  # Will map legend lines to original lines.
    for legline, origline in zip(leg.get_lines(), lines):
        legline.set_picker(True)  # Enable picking on the legend line.
        lined[legline] = origline
    
    def on_pick(event):
        # On the pick event, find the original line corresponding to the legend
        # proxy line, and toggle its visibility.
        legline = event.artist
        origline = lined[legline]
        visible = not origline.get_visible()
        origline.set_visible(visible)
        # Change the alpha on the line in the legend so we can see what lines
        # have been toggled.
        legline.set_alpha(1.0 if visible else 0.2)
        fig.canvas.draw()

    fig.canvas.mpl_connect('pick_event', on_pick)
    plt.show(block=False)

def plot_temperature_pressure_flowmeter(start_date:date, end_date:date):

    fig, (ax1, ax2, ax3) = plt.subplots(3,1, sharex=False)
    
    # temperature
    df_df_datetimes_all, df_temperatures_all = load_temperature(start_date, end_date)
    
    title = "Click on legend line to toggle line on/off" 
    ax1.set_title(title)
    ax1.set_xlabel('Datetime')
    ax1.set_ylabel('Temperature(K)')
    ax1.grid()
    
    line1, = ax1.plot(df_df_datetimes_all.iloc[:,0], df_temperatures_all.iloc[:,0], label="50 K")
    line2, = ax1.plot(df_df_datetimes_all.iloc[:,1], df_temperatures_all.iloc[:,1], label="4 K")
    line3, = ax1.plot(df_df_datetimes_all.iloc[:,2], df_temperatures_all.iloc[:,2], label="Still")
    line4, = ax1.plot(df_df_datetimes_all.iloc[:,3], df_temperatures_all.iloc[:,3], label="MCX")
    leg1 = ax1.legend(fancybox=True, shadow=True)
    
    # pressure
    df_datetimes_all, df_pressures_all = load_pressure(start_date, end_date)
   
    ax2.set_xlabel('Datetime')
    ax2.set_ylabel('Pressure (mBar)')
    ax2.grid()
    
    line11, = ax2.plot(df_datetimes_all, df_pressures_all.iloc[:,0], label="P1")
    line12, = ax2.plot(df_datetimes_all, df_pressures_all.iloc[:,1], label="P2")
    line13, = ax2.plot(df_datetimes_all, df_pressures_all.iloc[:,2], label="P3")
    line14, = ax2.plot(df_datetimes_all, df_pressures_all.iloc[:,3], label="P4")
    line15, = ax2.plot(df_datetimes_all, df_pressures_all.iloc[:,4], label="P5")
    line16, = ax2.plot(df_datetimes_all, df_pressures_all.iloc[:,5], label="P6")
    leg2 = ax2.legend(fancybox=True, shadow=True)

    # flowmeter
    df_datetimes_all, df_flowmeters_all = load_flowmeter(start_date, end_date)
    
    ax3.set_xlabel('Datetime')
    ax3.set_ylabel('Flowrate (mmol/s)')
    ax3.grid()
    
    line21, = ax3.plot(df_datetimes_all, df_flowmeters_all.iloc[:,0], label="Flowmeter")
    leg3 = ax3.legend(fancybox=True, shadow=True)

    plt.show(block=False)
    
def plot_temperature_pressure_flowmeter_status(start_date:date, end_date:date):

    fig, (ax1, ax2, ax3, ax4) = plt.subplots(4,1)
    
    # temperature
    df_df_datetimes_all, df_temperatures_all = load_temperature(start_date, end_date)
    
    title = "Click on legend line to toggle line on/off" 
    ax1.set_title(title)
    ax1.set_xlabel('Datetime')
    ax1.set_ylabel('Temperature(K)')
    ax1.grid()
    
    line1, = ax1.plot(df_df_datetimes_all.iloc[:,0], df_temperatures_all.iloc[:,0], label="50 K")
    line2, = ax1.plot(df_df_datetimes_all.iloc[:,1], df_temperatures_all.iloc[:,1], label="4 K")
    line3, = ax1.plot(df_df_datetimes_all.iloc[:,2], df_temperatures_all.iloc[:,2], label="Still")
    line4, = ax1.plot(df_df_datetimes_all.iloc[:,3], df_temperatures_all.iloc[:,3], label="MCX")
    # leg1 = ax1.legend(fancybox=True, shadow=True)
    
    # pressure
    df_datetimes_all, df_pressures_all = load_pressure(start_date, end_date)

    ax2.set_xlabel('Datetime')
    ax2.set_ylabel('Pressure (mBar)')
    ax2.grid()
    
    line11, = ax2.plot(df_datetimes_all, df_pressures_all.iloc[:,0], label="P1")
    line12, = ax2.plot(df_datetimes_all, df_pressures_all.iloc[:,1], label="P2")
    line13, = ax2.plot(df_datetimes_all, df_pressures_all.iloc[:,2], label="P3")
    line14, = ax2.plot(df_datetimes_all, df_pressures_all.iloc[:,3], label="P4")
    line15, = ax2.plot(df_datetimes_all, df_pressures_all.iloc[:,4], label="P5")
    line16, = ax2.plot(df_datetimes_all, df_pressures_all.iloc[:,5], label="P6")
    # leg2 = ax2.legend(fancybox=True, shadow=True)

    # flowmeter
    df_datetimes_all, df_flowmeters_all = load_flowmeter(start_date, end_date)
    
    ax3.set_xlabel('Datetime')
    ax3.set_ylabel('Flowrate (mmol/s)')
    ax3.grid()
    
    line21, = ax3.plot(df_datetimes_all, df_flowmeters_all.iloc[:,0], label="Flowmeter")
    # leg3 = ax3.legend(fancybox=True, shadow=True)

    # status
    df_datetimes_all, df_status_all = load_status(start_date, end_date)

    ax4.set_xlabel('Datetime')
    ax4.set_ylabel('Stauts')
    ax4.grid()

    s = "01-01-22,00:14:17,cptempwi,1.450000E+1,cptempwo,2.480000E+1,cptemph,7.050000E+1,cptempo,3.090000E+1,cpttime,2.548976E+6,cperrcode,2.800000E+1,cpavgl,8.840000E+1,cpavgh,2.748000E+2,nxdsf,3.000000E+1,nxdsct,4.100000E+1,nxdst,4.910500E+4,nxdsbs,4.050300E+4,nxdstrs,2.185700E+4,tc400remoteprio,1.000000E+0,tc400spdswptatt,1.000000E+0,tc400errorcode,0.000000E+0,tc400ovtempelec,0.000000E+0,tc400ovtemppump,0.000000E+0,tc400setspdatt,1.000000E+0,tc400pumpaccel,0.000000E+0,tc400heating,0.000000E+0,tc400standby,0.000000E+0,tc400pumpstatn,1.000000E+0,tc400commerr,0.000000E+0"
    legend_labels = s.split(',')[2::2]

    lines = []
    for i in range(24):
        line, = ax4.plot(df_datetimes_all, df_status_all.iloc[:,i], label=legend_labels[i])
        lines.append(line)

    leg = ax4.legend(fancybox=True, shadow=True)


    lined = {}  # Will map legend lines to original lines.
    for legline, origline in zip(leg.get_lines(), lines):
        legline.set_picker(True)  # Enable picking on the legend line.
        lined[legline] = origline
    
    def on_pick(event):
        # On the pick event, find the original line corresponding to the legend
        # proxy line, and toggle its visibility.
        legline = event.artist
        origline = lined[legline]
        visible = not origline.get_visible()
        origline.set_visible(visible)
        # Change the alpha on the line in the legend so we can see what lines
        # have been toggled.
        legline.set_alpha(1.0 if visible else 0.2)
        fig.canvas.draw()

        fig.canvas.mpl_connect('pick_event', on_pick)


    plt.show(block=False)

# Plot BlueFors log

In [6]:
start_date = date(2022,1,1)
end_date   = date(2022,3,1)

plot_temperature(start_date, end_date)
# plot_pressure(start_date, end_date)
# plot_flowmeter(start_date, end_date)
# plot_status(start_date, end_date)
# plot_temperature_pressure_flowmeter(start_date, end_date)
# plot_temperature_pressure_flowmeter_status(start_date, end_date)


FileNotFound: 2022-01-22, temperature
FileNotFound: 2022-01-23, temperature
FileNotFound: 2022-02-26, temperature
FileNotFound: 2022-02-27, temperature
FileNotFound: 2022-03-01, temperature


(                       0
 0    2022-01-01 00:00:48
 1    2022-01-01 00:01:50
 2    2022-01-01 00:02:50
 3    2022-01-01 00:03:52
 4    2022-01-01 00:04:52
 ...                  ...
 1425 2022-03-01 23:55:11
 1426 2022-03-01 23:56:11
 1427 2022-03-01 23:57:12
 1428 2022-03-01 23:58:12
 1429 2022-03-01 23:59:12
 
 [84979 rows x 1 columns],
             5       11     17     23      29     35
 0     0.000003  0.0337  402.0  424.0    8.71  1.220
 1     0.000003  0.0338  408.0  424.0    8.77  1.230
 2     0.000003  0.0338  405.0  424.0    8.76  1.230
 3     0.000003  0.0338  405.0  424.0    8.76  1.230
 4     0.000003  0.0338  408.0  424.0    8.71  1.230
 ...        ...     ...    ...    ...     ...    ...
 1425  0.000004  0.0233   19.0  169.0  728.00  0.509
 1426  0.000004  0.0233   19.0  168.0  727.00  0.509
 1427  0.000004  0.0233   19.0  169.0  728.00  0.510
 1428  0.000004  0.0233   19.0  168.0  728.00  0.509
 1429  0.000004  0.0233   19.0  168.0  728.00  0.510
 
 [84979 rows x 6 colu

In [2]:
matplotlib.get_backend() # show current backend

'module://matplotlib_inline.backend_inline'